In [54]:
import geopandas as gpd
import os
from sqlalchemy import create_engine
import pandas as pd

In [2]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [3]:
sql = '''
SELECT 
emergency_service_number AS ESN,
from_house_number AS LOW,
to_house_number AS HIGH,
street_name AS STREET,
street_type_code AS CODE,
city_name AS CITY,
Shape.STAsBinary() AS GEOM
FROM dbo.MSAG_Range;
'''

In [25]:
gdf = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='GEOM')

In [26]:
gdf.head()

,ESN,LOW,HIGH,STREET,CODE,CITY,GEOM
0,48,80485,80660,I-5,None,Cottage Grove,"POLYGON ((4256491.161 805646.915, 4256499.301 ..."
1,39,96406,96743,Lobster Creek,Rd,Alsea,"POLYGON ((4094864.378 963905.916, 4094870.933 ..."
2,39,96875,96949,Preacher Creek,Rd,Alsea,"POLYGON ((4078295.687 968282.069, 4078303.364 ..."
3,19,93490,93490,Amy,Rd,Blachly,"POLYGON ((4133706.637 935321.878, 4133697.954 ..."
4,12,20246,20264,Blachly Grange,Rd,Blachly,"POLYGON ((4121992.163 935443.878, 4121999.623 ..."


In [6]:
inpath = r'G:\projects\Address_Points\9-1-1_Net'

In [193]:
df = pd.read_excel(os.path.join(inpath, 'LANEORX2May2021.xls'), skiprows = [1])

In [194]:
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]

In [88]:
df.head()

,DI,STREET,LOW,HIGH,COMM,ST,O_E,ESN,DATE MODIFIED,EXCHANGE,ENTITY,MSAG,CODE
0,E,10TH,55,401,EUGENE,OR,B,1,2002-04-02,NaN,41,LANEOR,AVE
1,E,10TH,121,285,JUNCTION CITY,OR,B,5,1996-06-11,NaN,41,LANEOR,AVE
2,W,10TH,30,2103,EUGENE,OR,B,1,1996-06-11,NaN,41,LANEOR,AVE
3,W,10TH,190,1225,JUNCTION CITY,OR,B,5,1996-06-11,NaN,41,LANEOR,AVE
4,W,10TH,1361,1701,JUNCTION CITY,OR,B,17,2018-01-05,NaN,41,LANEOR,AVE


In [22]:
df.shape

(7993, 12)

In [23]:
# number of unique ESN in the Intrado extract
len(df.ESN.unique())

66

In [27]:
# number of unique ESN in the MSAG range
len(gdf.ESN.unique())

52

In [30]:
# number of common ESN in both
len([ESN for ESN in df.ESN.unique() if ESN in gdf.ESN.unique()])

51

In [31]:
# unique ESN that are not in the MSAG range
[ESN for ESN in df.ESN.unique() if ESN not in gdf.ESN.unique()]

[70, 58, 11, 60, 20, 72, 373, 372, 374, 375, 2372, 2373, 2374, 2375, 81]

In [33]:
# unique ESN that are not in the Intrado extract
[ESN for ESN in gdf.ESN.unique() if ESN not in df.ESN.unique()]

[39]

In [59]:
codes = list(gdf.CODE.unique())
codes.pop(0)
codes

['Rd',
 'Ln',
 'Dr',
 'St',
 'Hwy',
 'Ave',
 'Aly',
 'Way',
 'Ct',
 'Pl',
 'Lp',
 'Ter',
 'Blvd',
 'Cir',
 'Pkwy',
 'Pike']

In [65]:
codes = list(set(map(str.upper, codes)))

In [192]:
# split the code from address
def adjustAddress(x):
    code = x.split(' ')[-1]
    if code in codes:
        x=x.replace(' '+code, '')
    else:
        code=''
    return x, code

In [195]:
adjustAddress(df.STREET.unique()[1])[0]

'10TH'

In [196]:
df['CODE'] = df.STREET.apply(lambda x: adjustAddress(x)[1])
df['STREET'] = df.STREET.apply(lambda x: adjustAddress(x)[0])

In [100]:
gdf['STREET'] = gdf.STREET.str.upper()
gdf['CODE'] = gdf.CODE.str.upper()
gdf['CITY'] = gdf.CITY.str.upper()

In [103]:
# number of unique street in the MSAG range
len(gdf.STREET.unique())

3582

In [104]:
# number of unique street the Intrado extract
len(df.STREET.unique())

3635

In [102]:
# number of unique street in both
len([St for St in gdf.STREET.unique() if St in df.STREET.unique()])

3564

In [199]:
df.rename(columns={"COMM": "CITY"}, inplace = True)

In [110]:
gdf[(gdf.ESN==3) & (gdf.CITY=='COBURG') & (gdf.STREET=='RUSTIC')]

,ESN,LOW,HIGH,STREET,CODE,CITY,GEOM
144,3,91205,91239,RUSTIC,CT,COBURG,"POLYGON ((4249115.338 912085.878, 4249123.110 ..."


In [111]:
df[(df.ESN==3) & (df.CITY=='COBURG') & (df.STREET=='RUSTIC')]

,DI,STREET,LOW,HIGH,CITY,ST,O_E,ESN,DATE MODIFIED,EXCHANGE,ENTITY,MSAG,CODE
6726,NaN,RUSTIC,91205,91239,COBURG,OR,B,3,2014-09-22,NaN,41,LANEOR,CT


In [112]:
selected_columns = ['ESN', 'STREET', 'CODE', 'LOW', 'HIGH', 'CITY']

In [197]:
df1 = gdf.sort_values(by=selected_columns)[selected_columns] #, ignore_index=True

In [200]:
df2 = df.sort_values(by=selected_columns)[selected_columns]

In [202]:
df1.fillna("",inplace=True)

In [203]:
df1['KEY'] = df1.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' + str(row.LOW)  + '_' +  str(row.HIGH)  + '_' +  row.CITY, axis=1)

In [210]:
df1[df1['KEY']=='56_HWY 101__83570_83570_FLORENCE']

,ESN,STREET,CODE,LOW,HIGH,CITY,KEY
5213,56,HWY 101,,83570,83570,FLORENCE,56_HWY 101__83570_83570_FLORENCE
5214,56,HWY 101,,83570,83570,FLORENCE,56_HWY 101__83570_83570_FLORENCE


In [212]:
df1 = df1.drop_duplicates(subset=['KEY'])

In [213]:
df1.shape[0]

7910

In [215]:
df2['KEY'] = df2.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' + str(row.LOW)  + '_' +  str(row.HIGH)  + '_' +  row.CITY, axis=1)

In [216]:
df2.shape

(7993, 7)

In [217]:
len(df2.KEY.unique())

7993

In [219]:
df1.KEY.head()

986    1_10TH_ALY_535_1427_EUGENE
988     1_10TH_AVE_30_2103_EUGENE
987      1_10TH_AVE_55_401_EUGENE
989    1_10TH_PL_2620_2870_EUGENE
990     1_11TH_ALY_337_337_EUGENE
Name: KEY, dtype: object

In [221]:
df2.KEY.head()

7017    1_10TH_ALY_535_1427_EUGENE
2        1_10TH_AVE_30_2103_EUGENE
0         1_10TH_AVE_55_401_EUGENE
10      1_10TH_PL_2620_2870_EUGENE
7018     1_11TH_ALY_337_337_EUGENE
Name: KEY, dtype: object

In [227]:
# common keys in both
common_keys = [key for key in df1.KEY.unique() if key in df2.KEY.unique()]
len(common_keys)

7333

In [224]:
# keys from the MSAG range that are not in the Intrado extract
len([key for key in df1.KEY.unique() if key not in df2.KEY.unique()])

577

In [225]:
# keys from the Intrado extract that are not in the MSAG range
len([key for key in df2.KEY.unique() if key not in df1.KEY.unique()])

660

In [226]:
7333/7910

0.9270543615676359

In [228]:
def InCommon(x):
    if x in common_keys:
        res = 1
    else:
        res = 0
    return res

In [231]:
df1['D'] = df1.KEY.apply(lambda x: InCommon(x))

<ipython-input-231-e271b6a1cc2f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['D'] = df1.KEY.apply(lambda x: InCommon(x))


In [281]:
df3 = df1.loc[df1[df1.D==0].index,]

In [236]:
df2['D'] = df2.KEY.apply(lambda x: InCommon(x))

In [282]:
df4 = df2.loc[df2[df2.D==0].index,]

In [283]:
df3['KEYL'] = df3.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' + str(row.LOW) + '_' +  row.CITY, axis=1)

In [284]:
df4['KEYL'] = df4.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' + str(row.LOW) + '_' +  row.CITY, axis=1)

In [285]:
df3['KEYH'] = df3.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' + str(row.HIGH) + '_' +  row.CITY, axis=1)

In [286]:
df4['KEYH'] = df4.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' + str(row.HIGH) + '_' +  row.CITY, axis=1)

In [314]:
df3['KEYC'] = df3.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' +  row.CITY, axis=1)
df4['KEYC'] = df4.apply(lambda row: str(row.ESN) + '_' + row.STREET + '_' +  row.CODE + '_' +  row.CITY, axis=1)

In [315]:
len(df3.KEYC.unique())

500

In [292]:
df3.shape

(577, 10)

In [316]:
len(df4.KEYC.unique())

569

In [317]:
df4.shape

(660, 11)

In [295]:
df5 = df3.merge(df4, how='left', left_on='KEYL', right_on='KEYL')
df7 = df3.merge(df4, how='left', left_on='KEYH', right_on='KEYH')

In [296]:
df5.index = df3.index
df7.index = df3.index

In [306]:
df5.shape

(577, 19)

In [310]:
df5[df5.ESN_y.isna()]

,ESN_x,STREET_x,CODE_x,LOW_x,HIGH_x,CITY_x,KEY_x,D_x,KEYL,KEYH_x,ESN_y,STREET_y,CODE_y,LOW_y,HIGH_y,CITY_y,KEY_y,D_y,KEYH_y
1191,1,ALSEA,DR,1505,1595,EUGENE,1_ALSEA_DR_1505_1595_EUGENE,0,1_ALSEA_DR_1505_EUGENE,1_ALSEA_DR_1595_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1250,1,ARGON,AVE,711,991,EUGENE,1_ARGON_AVE_711_991_EUGENE,0,1_ARGON_AVE_711_EUGENE,1_ARGON_AVE_991_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,1,AUDEL,AVE,1286,1292,EUGENE,1_AUDEL_AVE_1286_1292_EUGENE,0,1_AUDEL_AVE_1286_EUGENE,1_AUDEL_AVE_1292_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,1,AUDEL,AVE,1297,1299,EUGENE,1_AUDEL_AVE_1297_1299_EUGENE,0,1_AUDEL_AVE_1297_EUGENE,1_AUDEL_AVE_1299_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1364,1,BARBER,DR,1020,1567,EUGENE,1_BARBER_DR_1020_1567_EUGENE,0,1_BARBER_DR_1020_EUGENE,1_BARBER_DR_1567_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7777,73,WINTER,LN,88613,88800,VENETA,73_WINTER_LN_88613_88800_VENETA,0,73_WINTER_LN_88613_VENETA,73_WINTER_LN_88800_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7778,73,WOLF CREEK,RD,23175,25829,VENETA,73_WOLF CREEK_RD_23175_25829_VENETA,0,73_WOLF CREEK_RD_23175_VENETA,73_WOLF CREEK_RD_25829_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7779,73,WOODBERRY,LN,87973,87995,VENETA,73_WOODBERRY_LN_87973_87995_VENETA,0,73_WOODBERRY_LN_87973_VENETA,73_WOODBERRY_LN_87995_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7780,73,WOODLAND,AVE,24913,25035,VENETA,73_WOODLAND_AVE_24913_25035_VENETA,0,73_WOODLAND_AVE_24913_VENETA,73_WOODLAND_AVE_25035_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [311]:
df7[df7.ESN_y.isna()]

,ESN_x,STREET_x,CODE_x,LOW_x,HIGH_x,CITY_x,KEY_x,D_x,KEYL_x,KEYH,ESN_y,STREET_y,CODE_y,LOW_y,HIGH_y,CITY_y,KEY_y,D_y,KEYL_y
1182,1,ALLEA,DR,1725,1755,EUGENE,1_ALLEA_DR_1725_1755_EUGENE,0,1_ALLEA_DR_1725_EUGENE,1_ALLEA_DR_1755_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1191,1,ALSEA,DR,1505,1595,EUGENE,1_ALSEA_DR_1505_1595_EUGENE,0,1_ALSEA_DR_1505_EUGENE,1_ALSEA_DR_1595_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,1,AUDEL,AVE,1286,1292,EUGENE,1_AUDEL_AVE_1286_1292_EUGENE,0,1_AUDEL_AVE_1286_EUGENE,1_AUDEL_AVE_1292_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1299,1,AUDEL,AVE,1297,1299,EUGENE,1_AUDEL_AVE_1297_1299_EUGENE,0,1_AUDEL_AVE_1297_EUGENE,1_AUDEL_AVE_1299_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1681,1,CENTENNIAL,LP,20,92,EUGENE,1_CENTENNIAL_LP_20_92_EUGENE,0,1_CENTENNIAL_LP_20_EUGENE,1_CENTENNIAL_LP_92_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7776,73,WILLOW,LN,22805,22839,VENETA,73_WILLOW_LN_22805_22839_VENETA,0,73_WILLOW_LN_22805_VENETA,73_WILLOW_LN_22839_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7777,73,WINTER,LN,88613,88800,VENETA,73_WINTER_LN_88613_88800_VENETA,0,73_WINTER_LN_88613_VENETA,73_WINTER_LN_88800_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7778,73,WOLF CREEK,RD,23175,25829,VENETA,73_WOLF CREEK_RD_23175_25829_VENETA,0,73_WOLF CREEK_RD_23175_VENETA,73_WOLF CREEK_RD_25829_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7779,73,WOODBERRY,LN,87973,87995,VENETA,73_WOODBERRY_LN_87973_87995_VENETA,0,73_WOODBERRY_LN_87973_VENETA,73_WOODBERRY_LN_87995_VENETA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [299]:
df6 = df4.merge(df3, how='left', left_on='KEYL', right_on='KEYL')
df8 = df4.merge(df3, how='left', left_on='KEYH', right_on='KEYH')

In [300]:
df6.index = df4.index
df8.index = df4.index

In [312]:
df6[df6.ESN_y.isna()]

,ESN_x,STREET_x,CODE_x,LOW_x,HIGH_x,CITY_x,KEY_x,D_x,KEYL,KEYH_x,ESN_y,STREET_y,CODE_y,LOW_y,HIGH_y,CITY_y,KEY_y,D_y,KEYH_y
603,1,8TH,AVE,888,1975,EUGENE,1_8TH_AVE_888_1975_EUGENE,0,1_8TH_AVE_888_EUGENE,1_8TH_AVE_1975_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7949,1,ALSEA DRIVE,,1505,1560,EUGENE,1_ALSEA DRIVE__1505_1560_EUGENE,0,1_ALSEA DRIVE__1505_EUGENE,1_ALSEA DRIVE__1560_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6824,1,AMAZON PARK,,1,99999,EUGENE,1_AMAZON PARK__1_99999_EUGENE,0,1_AMAZON PARK__1_EUGENE,1_AMAZON PARK__99999_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,1,ARGON,AVE,727,991,EUGENE,1_ARGON_AVE_727_991_EUGENE,0,1_ARGON_AVE_727_EUGENE,1_ARGON_AVE_991_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,1,AUDEL,AVE,1312,1316,EUGENE,1_AUDEL_AVE_1312_1316_EUGENE,0,1_AUDEL_AVE_1312_EUGENE,1_AUDEL_AVE_1316_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,375,WIRELESS CALL,,NaN,NaN,OAKRIDGE-EAST LANE E911,375_WIRELESS CALL__nan_nan_OAKRIDGE-EAST LANE ...,0,375_WIRELESS CALL__nan_OAKRIDGE-EAST LANE E911,375_WIRELESS CALL__nan_OAKRIDGE-EAST LANE E911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7400,2372,VOIP 911 CALL,,NaN,NaN,LANE-COMM CTR,2372_VOIP 911 CALL__nan_nan_LANE-COMM CTR,0,2372_VOIP 911 CALL__nan_LANE-COMM CTR,2372_VOIP 911 CALL__nan_LANE-COMM CTR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7401,2373,VOIP 911 CALL,,NaN,NaN,LANE-WEST,2373_VOIP 911 CALL__nan_nan_LANE-WEST,0,2373_VOIP 911 CALL__nan_LANE-WEST,2373_VOIP 911 CALL__nan_LANE-WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7402,2374,VOIP 911 CALL,,NaN,NaN,LANE-SOUTH,2374_VOIP 911 CALL__nan_nan_LANE-SOUTH,0,2374_VOIP 911 CALL__nan_LANE-SOUTH,2374_VOIP 911 CALL__nan_LANE-SOUTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
df8[df8.ESN_y.isna()]

,ESN_x,STREET_x,CODE_x,LOW_x,HIGH_x,CITY_x,KEY_x,D_x,KEYL_x,KEYH,ESN_y,STREET_y,CODE_y,LOW_y,HIGH_y,CITY_y,KEY_y,D_y,KEYL_y
601,1,8TH,AVE,35,861,EUGENE,1_8TH_AVE_35_861_EUGENE,0,1_8TH_AVE_35_EUGENE,1_8TH_AVE_861_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,1,ALLEA,DR,1725,1752,EUGENE,1_ALLEA_DR_1725_1752_EUGENE,0,1_ALLEA_DR_1725_EUGENE,1_ALLEA_DR_1752_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7949,1,ALSEA DRIVE,,1505,1560,EUGENE,1_ALSEA DRIVE__1505_1560_EUGENE,0,1_ALSEA DRIVE__1505_EUGENE,1_ALSEA DRIVE__1560_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6824,1,AMAZON PARK,,1,99999,EUGENE,1_AMAZON PARK__1_99999_EUGENE,0,1_AMAZON PARK__1_EUGENE,1_AMAZON PARK__99999_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,1,AUDEL,AVE,1312,1316,EUGENE,1_AUDEL_AVE_1312_1316_EUGENE,0,1_AUDEL_AVE_1312_EUGENE,1_AUDEL_AVE_1316_EUGENE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6974,375,WIRELESS CALL,,NaN,NaN,OAKRIDGE-EAST LANE E911,375_WIRELESS CALL__nan_nan_OAKRIDGE-EAST LANE ...,0,375_WIRELESS CALL__nan_OAKRIDGE-EAST LANE E911,375_WIRELESS CALL__nan_OAKRIDGE-EAST LANE E911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7400,2372,VOIP 911 CALL,,NaN,NaN,LANE-COMM CTR,2372_VOIP 911 CALL__nan_nan_LANE-COMM CTR,0,2372_VOIP 911 CALL__nan_LANE-COMM CTR,2372_VOIP 911 CALL__nan_LANE-COMM CTR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7401,2373,VOIP 911 CALL,,NaN,NaN,LANE-WEST,2373_VOIP 911 CALL__nan_nan_LANE-WEST,0,2373_VOIP 911 CALL__nan_LANE-WEST,2373_VOIP 911 CALL__nan_LANE-WEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7402,2374,VOIP 911 CALL,,NaN,NaN,LANE-SOUTH,2374_VOIP 911 CALL__nan_nan_LANE-SOUTH,0,2374_VOIP 911 CALL__nan_LANE-SOUTH,2374_VOIP 911 CALL__nan_LANE-SOUTH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [319]:
df9 = df3.drop_duplicates(subset=['KEYC'])
df10 = df4.drop_duplicates(subset=['KEYC'])

In [320]:
df11 = df9.merge(df10, how='left', left_on='KEYC', right_on='KEYC')

In [326]:
df11[~df11.ESN_y.isna()]

,ESN_x,STREET_x,CODE_x,LOW_x,HIGH_x,CITY_x,KEY_x,D_x,KEYL_x,KEYH_x,...,ESN_y,STREET_y,CODE_y,LOW_y,HIGH_y,CITY_y,KEY_y,D_y,KEYL_y,KEYH_y
0,1,8TH,AVE,35,1975,EUGENE,1_8TH_AVE_35_1975_EUGENE,0,1_8TH_AVE_35_EUGENE,1_8TH_AVE_1975_EUGENE,...,1.0,8TH,AVE,35,861,EUGENE,1_8TH_AVE_35_861_EUGENE,0.0,1_8TH_AVE_35_EUGENE,1_8TH_AVE_861_EUGENE
1,1,ALLEA,DR,1725,1755,EUGENE,1_ALLEA_DR_1725_1755_EUGENE,0,1_ALLEA_DR_1725_EUGENE,1_ALLEA_DR_1755_EUGENE,...,1.0,ALLEA,DR,1725,1752,EUGENE,1_ALLEA_DR_1725_1752_EUGENE,0.0,1_ALLEA_DR_1725_EUGENE,1_ALLEA_DR_1752_EUGENE
3,1,ARGON,AVE,711,991,EUGENE,1_ARGON_AVE_711_991_EUGENE,0,1_ARGON_AVE_711_EUGENE,1_ARGON_AVE_991_EUGENE,...,1.0,ARGON,AVE,727,991,EUGENE,1_ARGON_AVE_727_991_EUGENE,0.0,1_ARGON_AVE_727_EUGENE,1_ARGON_AVE_991_EUGENE
4,1,AUDEL,AVE,1286,1292,EUGENE,1_AUDEL_AVE_1286_1292_EUGENE,0,1_AUDEL_AVE_1286_EUGENE,1_AUDEL_AVE_1292_EUGENE,...,1.0,AUDEL,AVE,1312,1316,EUGENE,1_AUDEL_AVE_1312_1316_EUGENE,0.0,1_AUDEL_AVE_1312_EUGENE,1_AUDEL_AVE_1316_EUGENE
5,1,BARBER,DR,1020,1567,EUGENE,1_BARBER_DR_1020_1567_EUGENE,0,1_BARBER_DR_1020_EUGENE,1_BARBER_DR_1567_EUGENE,...,1.0,BARBER,DR,1015,1567,EUGENE,1_BARBER_DR_1015_1567_EUGENE,0.0,1_BARBER_DR_1015_EUGENE,1_BARBER_DR_1567_EUGENE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,73,PRAIRIE,RD,1765,1765,EUGENE,73_PRAIRIE_RD_1765_1765_EUGENE,0,73_PRAIRIE_RD_1765_EUGENE,73_PRAIRIE_RD_1765_EUGENE,...,73.0,PRAIRIE,RD,1675,1675,EUGENE,73_PRAIRIE_RD_1675_1675_EUGENE,0.0,73_PRAIRIE_RD_1675_EUGENE,73_PRAIRIE_RD_1675_EUGENE
458,73,SPROAT RANCH,RD,88310,88458,VENETA,73_SPROAT RANCH_RD_88310_88458_VENETA,0,73_SPROAT RANCH_RD_88310_VENETA,73_SPROAT RANCH_RD_88458_VENETA,...,73.0,SPROAT RANCH,RD,88310,88454,VENETA,73_SPROAT RANCH_RD_88310_88454_VENETA,0.0,73_SPROAT RANCH_RD_88310_VENETA,73_SPROAT RANCH_RD_88454_VENETA
467,73,TEMPLETON,RD,92965,92985,CHESHIRE,73_TEMPLETON_RD_92965_92985_CHESHIRE,0,73_TEMPLETON_RD_92965_CHESHIRE,73_TEMPLETON_RD_92985_CHESHIRE,...,73.0,TEMPLETON,RD,92965,93504,CHESHIRE,73_TEMPLETON_RD_92965_93504_CHESHIRE,0.0,73_TEMPLETON_RD_92965_CHESHIRE,73_TEMPLETON_RD_93504_CHESHIRE
470,73,TERRITORIAL,HWY,84572,86038,EUGENE,73_TERRITORIAL_HWY_84572_86038_EUGENE,0,73_TERRITORIAL_HWY_84572_EUGENE,73_TERRITORIAL_HWY_86038_EUGENE,...,73.0,TERRITORIAL,HWY,84572,86075,EUGENE,73_TERRITORIAL_HWY_84572_86075_EUGENE,0.0,73_TERRITORIAL_HWY_84572_EUGENE,73_TERRITORIAL_HWY_86075_EUGENE


In [325]:
df11[df11.ESN_y.isna()]

,ESN_x,STREET_x,CODE_x,LOW_x,HIGH_x,CITY_x,KEY_x,D_x,KEYL_x,KEYH_x,...,ESN_y,STREET_y,CODE_y,LOW_y,HIGH_y,CITY_y,KEY_y,D_y,KEYL_y,KEYH_y
2,1,ALSEA,DR,1505,1595,EUGENE,1_ALSEA_DR_1505_1595_EUGENE,0,1_ALSEA_DR_1505_EUGENE,1_ALSEA_DR_1595_EUGENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,CENTENNIAL,LP,20,92,EUGENE,1_CENTENNIAL_LP_20_92_EUGENE,0,1_CENTENNIAL_LP_20_EUGENE,1_CENTENNIAL_LP_92_EUGENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,CHESHIRE,AVE,200,200,EUGENE,1_CHESHIRE_AVE_200_200_EUGENE,0,1_CHESHIRE_AVE_200_EUGENE,1_CHESHIRE_AVE_200_EUGENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,1,CHESTNUT,DR,767,767,EUGENE,1_CHESTNUT_DR_767_767_EUGENE,0,1_CHESTNUT_DR_767_EUGENE,1_CHESTNUT_DR_767_EUGENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1,COUNTRY CLUB,PKWY,560,590,EUGENE,1_COUNTRY CLUB_PKWY_560_590_EUGENE,0,1_COUNTRY CLUB_PKWY_560_EUGENE,1_COUNTRY CLUB_PKWY_590_EUGENE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,73,WILLOW,LN,22805,22839,VENETA,73_WILLOW_LN_22805_22839_VENETA,0,73_WILLOW_LN_22805_VENETA,73_WILLOW_LN_22839_VENETA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
495,73,WINTER,LN,88613,88800,VENETA,73_WINTER_LN_88613_88800_VENETA,0,73_WINTER_LN_88613_VENETA,73_WINTER_LN_88800_VENETA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
496,73,WOLF CREEK,RD,23175,25829,VENETA,73_WOLF CREEK_RD_23175_25829_VENETA,0,73_WOLF CREEK_RD_23175_VENETA,73_WOLF CREEK_RD_25829_VENETA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
497,73,WOODBERRY,LN,87973,87995,VENETA,73_WOODBERRY_LN_87973_87995_VENETA,0,73_WOODBERRY_LN_87973_VENETA,73_WOODBERRY_LN_87995_VENETA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [324]:
193/500

0.386